In [65]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from load_data_gpu import UrbanSoundDataset
import torchaudio
from cnn import CNNNetwork

In [66]:
# Steup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [67]:
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader

In [68]:
ANNOTATIONS_FILE = r"E:\DL_audio\with_pytorch\data1\metadata\fold_1_annotation.csv"
AUDIO_DIR = r"E:\DL_audio\with_pytorch\data1\audio"

SAMPLE_RATE=22050
NUM_SAMPLES = 22050

mel_spec = torchaudio.transforms.MelSpectrogram(sample_rate = SAMPLE_RATE,
                                              n_fft = 1024,
                                              hop_length = 512,
                                              n_mels = 64)
    
usd = UrbanSoundDataset(ANNOTATIONS_FILE,AUDIO_DIR,mel_spec,SAMPLE_RATE,NUM_SAMPLES,device)

signal , label = usd[0]

print(signal) , print(label) , print(signal.shape), print(signal.device)

tensor([[[1.7433e-03, 1.6207e-04, 4.0781e-04,  ..., 2.0526e-03,
          6.8678e-03, 1.7846e-02],
         [6.4027e-04, 5.4754e-03, 2.8272e-02,  ..., 5.5625e-03,
          8.6759e-02, 5.2350e-02],
         [1.0592e-03, 1.2941e-02, 8.1117e-02,  ..., 5.6558e-03,
          6.1651e-02, 1.0182e-01],
         ...,
         [4.8316e-06, 2.0553e-06, 2.5774e-06,  ..., 3.5377e-06,
          3.0520e-06, 4.1328e-06],
         [4.0104e-06, 4.2231e-06, 4.8558e-06,  ..., 1.7981e-06,
          3.2201e-06, 6.0627e-06],
         [4.0844e-06, 4.0532e-06, 4.6760e-06,  ..., 1.6818e-06,
          3.3693e-06, 2.2788e-06]]], device='cuda:0')
3
torch.Size([1, 64, 44])
cuda:0


(None, None, None, None)

train = 0.8*len(usd)

train_data = usd[:train]
test_data = usd[train:]

In [69]:
# Create a dataloader
Batch=128

train_data_loader = create_data_loader(usd, batch_size=Batch)
#test_data_loader = create_data_loader(test_data, batch_size=Batch)

In [70]:
model_0 = CNNNetwork().to(device)
print(model_0)

CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [71]:
def accuracy_fn(y_true,y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred))*100
    return acc

In [72]:
def train_step(model, data_loader, loss_fn, accuracy_fn, optimizer, device):
    
    model.train()
    for X,y in data_loader:
        X , y = X.to(device) , y.to(device)
        
        y_logits = model(X).squeeze()
        y_preds = torch.argmax(torch.round(torch.sigmoid(y_logits)),dim=1)
        
        #print(y_preds) , print(y)
        
        loss = loss_fn(y_logits,y)
        acc = accuracy_fn(y,y_preds)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    print(f"Loss: {loss}| Acc: {acc}")
    

def test_step(model, data_loader, loss_fn, device):
    
    model.eval()
    
    with torch.inference_mode():
        
        for X,y in data_loader:
            X , y = X.to(device) , y.to(device)

            test_pred = model(X)

            test_loss = loss_fn(test_pred,y)

        
    print(f"Test Loss: {test_loss}")

In [73]:
LEARNING_RATE = 0.001

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=LEARNING_RATE,
                           weight_decay=0.0001)


In [74]:
# Set the number of epochs
epochs = 500

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch} \n-----------")
    ### Training
    train_step(model=model_0,
                   data_loader=train_data_loader,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   accuracy_fn=accuracy_fn,
                   device=device)
    
      

#       ###testing
#       test_step(model=model_0,
#                    data_loader=test_data_loader,
#                    loss_fn=loss_fn,
#                    device=device)

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0 
-----------
Loss: 2.3567733764648438| Acc: 2.857142857142857
Epoch: 1 
-----------
Loss: 2.2933666706085205| Acc: 2.857142857142857
Epoch: 2 
-----------
Loss: 2.2402215003967285| Acc: 8.571428571428571
Epoch: 3 
-----------
Loss: 2.2324366569519043| Acc: 8.571428571428571
Epoch: 4 
-----------
Loss: 2.229637861251831| Acc: 9.523809523809524
Epoch: 5 
-----------
Loss: 2.2306830883026123| Acc: 9.523809523809524
Epoch: 6 
-----------
Loss: 2.2279741764068604| Acc: 10.476190476190476
Epoch: 7 
-----------
Loss: 2.2274060249328613| Acc: 11.428571428571429
Epoch: 8 
-----------
Loss: 2.2267279624938965| Acc: 11.428571428571429
Epoch: 9 
-----------
Loss: 2.224585771560669| Acc: 11.428571428571429
Epoch: 10 
-----------
Loss: 2.2234115600585938| Acc: 11.428571428571429
Epoch: 11 
-----------
Loss: 2.222585439682007| Acc: 11.428571428571429
Epoch: 12 
-----------
Loss: 2.219367504119873| Acc: 11.428571428571429
Epoch: 13 
-----------
Loss: 2.218005895614624| Acc: 11.428571428571429

Loss: 1.9353374242782593| Acc: 25.71428571428571
Epoch: 115 
-----------
Loss: 1.9382078647613525| Acc: 29.523809523809526
Epoch: 116 
-----------
Loss: 1.934908151626587| Acc: 28.57142857142857
Epoch: 117 
-----------
Loss: 1.9380221366882324| Acc: 30.476190476190478
Epoch: 118 
-----------
Loss: 1.9360750913619995| Acc: 27.61904761904762
Epoch: 119 
-----------
Loss: 1.968184232711792| Acc: 26.666666666666668
Epoch: 120 
-----------
Loss: 1.9502816200256348| Acc: 31.428571428571427
Epoch: 121 
-----------
Loss: 1.939582347869873| Acc: 31.428571428571427
Epoch: 122 
-----------
Loss: 1.9304579496383667| Acc: 32.38095238095238
Epoch: 123 
-----------
Loss: 1.9329776763916016| Acc: 32.38095238095238
Epoch: 124 
-----------
Loss: 1.9354814291000366| Acc: 26.666666666666668
Epoch: 125 
-----------
Loss: 1.9412636756896973| Acc: 30.476190476190478
Epoch: 126 
-----------
Loss: 1.9345484972000122| Acc: 28.57142857142857
Epoch: 127 
-----------
Loss: 1.931901454925537| Acc: 29.52380952380952

Loss: 1.888165831565857| Acc: 29.523809523809526
Epoch: 227 
-----------
Loss: 1.8872971534729004| Acc: 33.33333333333333
Epoch: 228 
-----------
Loss: 1.8895806074142456| Acc: 31.428571428571427
Epoch: 229 
-----------
Loss: 1.8965202569961548| Acc: 40.0
Epoch: 230 
-----------
Loss: 1.9154558181762695| Acc: 30.476190476190478
Epoch: 231 
-----------
Loss: 1.8922851085662842| Acc: 34.285714285714285
Epoch: 232 
-----------
Loss: 1.885780930519104| Acc: 30.476190476190478
Epoch: 233 
-----------
Loss: 1.9010943174362183| Acc: 27.61904761904762
Epoch: 234 
-----------
Loss: 1.8865224123001099| Acc: 31.428571428571427
Epoch: 235 
-----------
Loss: 1.88616943359375| Acc: 30.476190476190478
Epoch: 236 
-----------
Loss: 1.879544734954834| Acc: 33.33333333333333
Epoch: 237 
-----------
Loss: 1.8746131658554077| Acc: 34.285714285714285
Epoch: 238 
-----------
Loss: 1.8767870664596558| Acc: 35.23809523809524
Epoch: 239 
-----------
Loss: 1.8760435581207275| Acc: 36.19047619047619
Epoch: 240 


Loss: 1.8588248491287231| Acc: 36.19047619047619
Epoch: 340 
-----------
Loss: 1.8584418296813965| Acc: 36.19047619047619
Epoch: 341 
-----------
Loss: 1.8584568500518799| Acc: 36.19047619047619
Epoch: 342 
-----------
Loss: 1.8583500385284424| Acc: 35.23809523809524
Epoch: 343 
-----------
Loss: 1.8583670854568481| Acc: 36.19047619047619
Epoch: 344 
-----------
Loss: 1.8587467670440674| Acc: 34.285714285714285
Epoch: 345 
-----------
Loss: 1.8583056926727295| Acc: 36.19047619047619
Epoch: 346 
-----------
Loss: 1.857546091079712| Acc: 37.142857142857146
Epoch: 347 
-----------
Loss: 1.8578486442565918| Acc: 36.19047619047619
Epoch: 348 
-----------
Loss: 1.8571549654006958| Acc: 37.142857142857146
Epoch: 349 
-----------
Loss: 1.8602761030197144| Acc: 33.33333333333333
Epoch: 350 
-----------
Loss: 1.8651400804519653| Acc: 36.19047619047619
Epoch: 351 
-----------
Loss: 1.8578656911849976| Acc: 35.23809523809524
Epoch: 352 
-----------
Loss: 1.877032995223999| Acc: 33.33333333333333
E

Loss: 1.885486364364624| Acc: 41.904761904761905
Epoch: 454 
-----------
Loss: 1.884893536567688| Acc: 40.95238095238095
Epoch: 455 
-----------
Loss: 1.9246399402618408| Acc: 37.142857142857146
Epoch: 456 
-----------
Loss: 1.8852657079696655| Acc: 40.95238095238095
Epoch: 457 
-----------
Loss: 1.8839225769042969| Acc: 42.857142857142854
Epoch: 458 
-----------
Loss: 1.8829610347747803| Acc: 41.904761904761905
Epoch: 459 
-----------
Loss: 1.8826664686203003| Acc: 41.904761904761905
Epoch: 460 
-----------
Loss: 1.8838987350463867| Acc: 40.0
Epoch: 461 
-----------
Loss: 1.8841407299041748| Acc: 40.95238095238095
Epoch: 462 
-----------
Loss: 1.8838634490966797| Acc: 40.0
Epoch: 463 
-----------
Loss: 1.8833287954330444| Acc: 40.0
Epoch: 464 
-----------
Loss: 1.8833354711532593| Acc: 40.0
Epoch: 465 
-----------
Loss: 1.8831032514572144| Acc: 40.0
Epoch: 466 
-----------
Loss: 1.8830033540725708| Acc: 40.0
Epoch: 467 
-----------
Loss: 1.8829034566879272| Acc: 40.0
Epoch: 468 
-----

In [75]:
model_0.state_dict()

OrderedDict([('conv1.0.weight',
              tensor([[[[ 0.1262, -0.2496, -0.0866],
                        [ 0.0948,  0.1738, -0.2108],
                        [-0.0852, -0.0400, -0.2270]]],
              
              
                      [[[-0.0211, -0.2082, -0.0007],
                        [-0.3153,  0.1820, -0.2415],
                        [ 0.2751,  0.2665, -0.0075]]],
              
              
                      [[[ 0.2520, -0.2293,  0.1595],
                        [ 0.1830, -0.1351, -0.2217],
                        [-0.2916,  0.3143, -0.1490]]],
              
              
                      [[[ 0.1632, -0.2727, -0.0650],
                        [ 0.2939,  0.2563, -0.2404],
                        [ 0.2418,  0.0488, -0.2206]]],
              
              
                      [[[-0.1020, -0.1028, -0.0249],
                        [-0.1314,  0.0839, -0.0338],
                        [ 0.0635,  0.0525, -0.3277]]],
              
              
             

In [76]:
torch.save(model_0.state_dict(), "cnnnet.pth")
print("Model stored")

Model stored
